# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [ ]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [ ]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

In [ ]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """
    applicazione_pca = False
    pca_standardize = False
    
    if hyperparams['use_pca']:
        applicazione_pca = True
        pca_standardize = hyperparams['pca_standardize']

    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    if applicazione_pca and pca_standardize and hyperparams['data_standardize'] == False:
        scaler_pca = StandardScaler()
        X_train = scaler_pca.fit_transform(X_train)
        X_val = scaler_pca.transform(X_val)
        
    if applicazione_pca:
        pca = PCA(n_components=hyperparams['pca_components'])
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)


    # Aggiunge il termine costante ai dati

    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train]
    X_val = np.c_[np.ones((X_val.shape[0],1)), X_val]

 # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        reg_lambda = hyperparams['reg_lambda']
        I = np.eye(X_train.shape[1])
        I[0,0] = 0
        parametri = np.linalg.pinv(X_train.T @ X_train + reg_lambda * I) @ X_train.T @ y_train
        
    else:
        parametri = np.linalg.pinv(X_train.T @ X_train) @ X_train.T @ y_train


    # Calcolo predizioni

    y_val_predizioni = X_val @ parametri
    

    # Calcola il MAE
    mae_val = np.mean(np.abs(y_val - y_val_predizioni))

    return mae_val

# Dividi il dataset in training e test set
Train_porzione = 0.6
test_porzione = 0.4

num_Train = int(Train_porzione * X.shape[0])
num_test = int(test_porzione * X.shape[0])

X_Train = X[:num_Train]
y_Train = y[:num_Train]

X_test = X[num_Train:]
y_test = y[num_Train:]

# Dividi il training set in training set effettivo e validation set

effettivo_porzione = 0.5
validation_porzione = 0.5

num_train = int(effettivo_porzione * X_Train.shape[0])
num_validation = int(validation_porzione * X_Train.shape[0])

X_train = X_Train[:num_train]
y_train = y_Train[:num_train]

X_val = X_Train[num_train:]
y_val = y_Train[num_train:]


# Trova la configurazione di iperparametri migliore
def iper_migliori(configs, X_train, y_train, X_val, y_val):
    config_migliore = None
    mae_migliore = float('inf')
    
    for config in configs:
        mae_val = pipeline(X_train, y_train, X_val, y_val, config)

        if mae_val < mae_migliore:
            mae_migliore = mae_val
            config_migliore = config
            
    return mae_migliore, config_migliore

# Riallena il modello sul training set completo
mae_migliore, config_migliore = iper_migliori(configs, X_train, y_train, X_val, y_val)

mae_migliore_traing_completo = pipeline(X_Train.copy(), y_Train.copy(), X_Train.copy(), y_Train.copy(), config_migliore)
  

# Calcola il MAE sul test set

mae_migliore_test = pipeline(X_Train, y_Train, X_test, y_test, config_migliore)



# Stampa  risultati
print(config_migliore)
print(mae_migliore)

print(mae_migliore_traing_completo)
print(mae_migliore_test)

In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 